In [ ]:
import numpy as np
import nglview as nv
import MDAnalysis as mda
from MDAnalysis.transformations import translate, rotateby, center_in_box
import prolif as plf
from rdkit.Geometry import Point3D
from ipywidgets import interactive, HBox, Layout,VBox
from pprint import pprint

In [ ]:
u1 = mda.Universe(plf.datafiles.datapath / "benzene.mol2")
elements = mda.topology.guessers.guess_types(u1.atoms.names)
u1.add_TopologyAttr("elements", elements)
u1.segments.segids = np.array(["U1"], dtype=object)
u1.transfer_to_memory()

def create(xyz=[0, 0, 0], rotation=[0,0,0]):
    u2 = u1.copy()
    u2.segments.segids = np.array(["U2"], dtype=object)
    tr = translate(xyz)
    rotx = rotateby(rotation[0], [1,0,0], ag=u2.atoms)
    roty = rotateby(rotation[1], [0,1,0], ag=u2.atoms)
    rotz = rotateby(rotation[2], [0,0,1], ag=u2.atoms)
    u2.trajectory.add_transformations(tr, rotx, roty, rotz)
    u2.transfer_to_memory()
    u = mda.Merge(u1.atoms, u2.atoms)
    return u

In [ ]:
fp = plf.Fingerprint()
rad90 = np.pi/2

def cap_angle(angle, cap=rad90):
    if angle >= np.pi:
        angle %= cap
    elif angle > cap:
        angle = (cap - (angle % cap))
    return angle

def measure(u):
    ag1 = u.select_atoms("segid U1")
    ring1 = ag1.select_atoms("type C.2").positions.astype(float)
    c1 = plf.utils.get_centroid(ring1)
    c1 = Point3D(*c1)
    n1 = plf.utils.get_ring_normal_vector(c1, ring1)
    
    ag2 = u.select_atoms("segid U2")
    ring2 = ag2.select_atoms("type C.2").positions.astype(float)
    c2 = plf.utils.get_centroid(ring2)
    c2 = Point3D(*c2)
    n2 = plf.utils.get_ring_normal_vector(c2, ring2)
    
    planes_angle = n1.AngleTo(n2)
    c1c2 = c1.DirectionVector(c2)
    c2c1 = c2.DirectionVector(c1)
    n1c1c2 = n1.AngleTo(c1c2)
    n2c2c1 = n2.AngleTo(c2c1)
    above = (plf.utils.angle_between_limits(n1c1c2, 0, np.radians(40), ring=True)
             or
             plf.utils.angle_between_limits(n2c2c1, 0, np.radians(40), ring=True))
    
    m1 = plf.Molecule.from_mda(ag1)
    m2 = plf.Molecule.from_mda(ag2)
        
    print(f'''
centroid distance: {c1.Distance(c2):.3f}
planes: {np.degrees(cap_angle(planes_angle)):.3f}°   above: {above}
n1c1c2: {np.degrees(cap_angle(n1c1c2)):.3f}°   n2c2c1: {np.degrees(cap_angle(n2c2c1)):.3f}°
FTF: {fp.facetoface(m1, m2)}   ETF: {fp.edgetoface(m1, m2)}''')
    return c1, n1, c2, n2

In [ ]:
u = create(xyz=[0, 1.5, 4.5], rotation=[30, 0, 0])
v = nv.show_mdanalysis(u.atoms)
v.center("*")
v._set_size("100%", "400px")
v.camera = 'orthographic'
shapes = {}

def view(dx=0, dy=1.5, dz=4.5, ax=30, ay=0, az=0):
    new = create(xyz=[dx, dy, dz], rotation=[ax, ay, az])
    u.atoms.positions = new.atoms.positions
    v.set_coordinates({0: new.atoms.positions})
    c1, n1, c2, n2 = measure(u)
    try:
        for comp in shapes.values():
            comp.clear()
    except:
        pass
    shapes["c1c2"] = v.shape.add_cylinder(list(c1), list(c2), [1,0,0], .1)
    shapes["n1"] = v.shape.add_cylinder(list(c1), list(c1 + n1 + n1), [0,1,0], .1)
    shapes["n2"] = v.shape.add_cylinder(list(c2), list(c2 + n2 + n2), [0,0,1], .1)
        
widget=interactive(view, 
                   dx=(-7, 7, .5), dy=(-7, 7, .5), dz=(-7, 7, .5),
                   ax=(0, 180, 5), ay=(0, 180, 5), az=(0, 180, 5))
controls = HBox(widget.children[:-1], layout=Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output, v]))